In [ ]:
!pip install transformers

In [ ]:
# Importar bibliotecas necesarias
import re
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
import torch
from sklearn.metrics import classification_report, confusion_matrix

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Cargar desde Google Drive
df_subset = pd.read_csv('/content/drive/My Drive/NLP/datos_explorados.csv')

In [ ]:
# Crear la columna de etiquetas basada en el rating
df_subset['label'] = (df_subset['rating'] >= 4).astype(int)

# Dividir los datos en características (X) y etiquetas (y)
X = df_subset['text']
y = df_subset['label']

# Dividir en conjunto de entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

Carga el modelo y el tokenizador:

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Dmyadav2001/Sentimental-Analysis")
model = AutoModelForSequenceClassification.from_pretrained("Dmyadav2001/Sentimental-Analysis")

Ajustar el modelo para clasificación binaria

In [ ]:
model.classifier = torch.nn.Linear(model.classifier.in_features, 2)
model.num_labels = 2

Tokenización de datos

In [ ]:
train_encodings = tokenizer(list(X_train), truncation=True, padding=True)
test_encodings = tokenizer(list(X_test), truncation=True, padding=True)

Crear datasets de PyTorch

In [ ]:
class SentimentDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

train_dataset = SentimentDataset(train_encodings, y_train.tolist())
test_dataset = SentimentDataset(test_encodings, y_test.tolist())


Configurar el entrenamiento

In [ ]:
training_args = TrainingArguments(
    output_dir='/content/drive/MyDrive/NLP/results',
    num_train_epochs=3,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=64,
    warmup_steps=500,
    weight_decay=0.01,
    logging_dir='/content/drive/MyDrive/NLP/logs',
    evaluation_strategy="epoch",
    report_to="none",
    save_strategy="epoch",  # Guardar modelo al final de cada época
    load_best_model_at_end=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1568: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


Entrenar el modelo

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,0.270498
2,0.244300,0.228160
3,0.151100,0.329376


TrainOutput(global_step=1200, training_loss=0.1772159186999003, metrics={'train_runtime': 956.7178, 'train_samples_per_second': 20.069, 'train_steps_per_second': 1.254, 'total_flos': 2543419411660800.0, 'train_loss': 0.1772159186999003, 'epoch': 3.0})

Evaluar el modelo

In [ ]:
eval_results = trainer.evaluate()
print("Resultados de la evaluación:")
print(eval_results)

Resultados de la evaluación:
{'eval_loss': 0.22816045582294464, 'eval_runtime': 23.001, 'eval_samples_per_second': 69.562, 'eval_steps_per_second': 1.087, 'epoch': 3.0}


Obtener predicciones en el conjunto de prueba

In [ ]:
predictions = trainer.predict(test_dataset)
predicted_labels = np.argmax(predictions.predictions, axis=1)

Imprimir el informe de clasificación y la matriz de confusión

In [ ]:
print("\nInforme de Clasificación:")
print(classification_report(y_test, predicted_labels))
print("\nMatriz de Confusión:")
print(confusion_matrix(y_test, predicted_labels))


Informe de Clasificación:
              precision    recall  f1-score   support

           0       0.91      0.94      0.92       800
           1       0.94      0.91      0.92       800

    accuracy                           0.92      1600
   macro avg       0.92      0.92      0.92      1600
weighted avg       0.92      0.92      0.92      1600


Matriz de Confusión:
[[750  50]
 [ 76 724]]


El modelo presenta un rendimiento sólido y equilibrado en la clasificación de las dos clases, con una precisión de 0.91 para la clase 0 y 0.94 para la clase 1, lo que indica alta exactitud en las predicciones. El recall es igualmente positivo, alcanzando 0.94 para la clase 0 y 0.91 para la clase 1, lo que refleja una buena capacidad para identificar correctamente los casos reales. Con un F1-score de 0.92 y 0.92 respectivamente, el modelo muestra un buen equilibrio entre precisión y recall. La accuracy global del 92% sugiere que el modelo clasifica correctamente la mayoría de las muestras, la matriz de confusión revela una distribución equilibrada de verdaderos positivos y negativos, confirmando así el rendimiento efectivo del modelo en ambas clases.